## Data Mining project: Discover and describe areas of interest an events from geo-located data

## Task #1: Import Dataset and Librarie

## Library Installation

In [ ]:
# installation of required libraries and dependencies
# numeric calculations
! pip install numpy==1.26.0 
# data frames 
! pip install pandas==2.1.1 
# machine learning algorithms 
! pip install scikit-learn==1.5.1 
! pip install scipy==1.12.0
# plotting 
! pip install plotly==5.24.1 
! pip install matplotlib==3.8.0 
! pip install seaborn==0.13.2 
! pip install plotly-express==0.4.1 
! pip install chart-studio==1.1.0 
# web app library 
! pip install streamlit==1.37.1 
# association rules
! pip install mlxtend==0.23.3 
! pip install folium==0.19.4
! pip install geopy==2.4.1
! pip install pyproj
# text mining
! pip install nltk==3.6.5
! pip install wordcloud
! pip install pillow==9.5.0
! pip install osmnx


## Import

In [ ]:
# load pandas to deal with the data
import pandas as pd
# plotting
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.colors as mcolors
from geopy.distance import geodesic
from scipy.spatial import ConvexHull, QhullError

from sklearn.cluster import AgglomerativeClustering,  DBSCAN  ,linkage_tree  
from scipy.cluster.hierarchy import dendrogram
import folium
import seaborn as sns
import random
from scipy.spatial.distance import cdist
from itertools import permutations
import osmnx as ox
import networkx as nx

# Data cleaning and preparation

In [ ]:
# Charger le fichier CSV
data = pd.read_csv("../data/dataset.csv", sep=",", low_memory=False)

data.info()

### Type Conversion

In [ ]:
# Sélectionner les colonnes 11 et 12 (indices 10 et 11 en pandas)
columns_to_check = data.iloc[:, [11, 12]]

# Parcourir chaque colonne sélectionnée
for col in columns_to_check.columns:
    print(f"Analyse de la colonne: {col}")
    
    # Trouver les types de données présents
    types_present = data[col].map(type).value_counts()
    print("Types présents dans cette colonne :")
    print(types_present)
    
    # Afficher l'utilisation de la mémoire pour cette colonne
    memory_usage = data[col].memory_usage(deep=True)
    print(f"Utilisation de la mémoire pour cette colonne : {memory_usage} bytes\n")

for col in columns_to_check.columns:
    print(f"Conversion de la colonne: {col}")
    data[col] = pd.to_numeric(data[col], errors='coerce').fillna(0)  # Convertir en float, remplacer les erreurs par NaN

    # Vérifier le résultat
    print(f"Colonne {col} après conversion:")
    print(data[col].head())  # Afficher les premières lignes pour valider
    print(f"Types après conversion: {data[col].map(type).value_counts()}\n")

In [ ]:
#rename columms
data.columns = data.columns.str.strip()
print("Colonnes après nettoyage :", data.columns.tolist())

### NULL Values

In [ ]:
#Find null values
data.isna().sum()

In [ ]:
print(f"Initial: {len(data)}")
print(data.columns)
columns_to_clean = [
					'lat', 'long', 'date_taken_minute', 'date_taken_hour', 
					'date_taken_day', 'date_taken_month', 'date_taken_year', 
					'date_upload_minute', 'date_upload_hour', 'date_upload_day', 
					'date_upload_month', 'date_upload_year'
					]
data_cleaned = data.dropna(subset=columns_to_clean)
print(f"After removing missing values: {len(data_cleaned)}")

### Duplicates

In [ ]:
# Check if there are any duplicates
data_cleaned.duplicated().sum()

In [ ]:
# Remove duplicates if necessary
print(f"Initial: {len(data_cleaned)}")
data_cleaned = data_cleaned.drop_duplicates(subset=['user','lat','long','title','date_taken_year','date_taken_month','date_taken_day'],keep='first')
print(f"After removing duplicates: {len(data_cleaned)}") 

### Columns

Note that the data with a non-null values at column 16 is sorted differently. We conclude that it is hard to determine which value belongs to which column since date_taken_minute, date_taken_month can be ambigous.

In [ ]:
columns = ['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18']
column_data = data_cleaned.dropna(subset=columns, how='all')

print(column_data.iloc[0])

The rows that have non-null values in columns 16, 17 and 18 are retracted from the dataset. These columns thus serve no purpose and are removed as well.

In [ ]:
# Specify the columns to check for non-null values
columns_to_check = ['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18']

# Drop rows where any of the specified columns are not null
print(f"Initial: {len(data_cleaned)}")
data_cleaned = data_cleaned[data_cleaned[columns_to_check].isnull().all(axis=1)]
print(f"After removing non-null values: {len(data_cleaned)}")

# Drop the columns
data_cleaned = data_cleaned.drop(columns=columns)

print(data_cleaned.columns)

### Save Cleaned Data

In [ ]:
data_cleaned.to_csv("../data/data-cleaned.csv", index=False)

## Descriptive Statistics

In [ ]:
data_cleaned.describe(exclude=[object])

## Data Visualisation

### Geolocalisation Marker

In [ ]:
m = folium.Map(location=(45.764, 4.8357), zoom_start=12)

# Sample a subset of the data (e.g., 500 points)
sampled_data = data_cleaned.sample(n=1000, random_state=1)

latitude_col = sampled_data['lat']
longitude_col = sampled_data['long']

for i in range(0, len(sampled_data)):
	folium.Marker([latitude_col.iloc[i], longitude_col.iloc[i]]).add_to(m)

m

### Geolocalisation Heatmap

In [ ]:
from folium.plugins import HeatMap

m = folium.Map(location=(45.764, 4.8357), zoom_start=12)

# Sample a subset of the data (e.g., 500 points)
sampled_data = data_cleaned.sample(n=1000, random_state=1)

latitude_col = sampled_data['lat']
longitude_col = sampled_data['long']

heat_data = [[row['lat'], row['long']] for index, row in sampled_data.iterrows()]
HeatMap(heat_data).add_to(m)

m

## Clustering

### K-Means

The only data necessary to create clusters here would be the latitude and longitude. We'll drop all other columns as it is not necessary.

In [ ]:
# Specify the columns to keep
columns = ['lat', 'long']

# Drop all columns except the specified ones
df_clustering = data_cleaned[columns]
print(df_clustering.head())

Although the longitude and latitude have comparable scales, we can use the StandardScaler to normalize the data

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_clustering)
scaled_data_df = pd.DataFrame(data=scaled_data, columns=df_clustering.columns)

plt.figure(figsize=(10, 6))
plt.scatter(scaled_data_df['lat'], scaled_data_df['long'], alpha=0.5)
plt.title('Scaled Latitude and Longitude')
plt.xlabel('Scaled Latitude')
plt.ylabel('Scaled Longitude')
plt.grid(True)
plt.show()

Interestingly, the same pattern observed in the heatmap is reproduced with the scaled data. This is expected, as the data has been proportionally scaled

We'll find the optimal number of clusters using Elbow method

In [ ]:
# Range of k values to try
k_values = range(1, 40)
sum_square = []

for k in k_values:
	kmeans = KMeans(n_clusters=k, init='k-means++')
	kmeans.fit(scaled_data_df)
	sum_square.append(kmeans.inertia_)

plt.plot(k_values, sum_square)
plt.title('Elbow Method')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Sum square')
plt.show()


The optimal number of clusters can be interpreted to be between 6-10. We found that with 6-10 clusters, the granularity of places is not as well defined as the number of points are extremely vast and spreaded.

In [ ]:
kmeans = KMeans(n_clusters=70, init='k-means++')
kmeans.fit(scaled_data_df)
data_cleaned['cluster_kmeans'] = kmeans.labels_ 
print(data_cleaned[['cluster_kmeans', 'lat', 'long']])

In [ ]:
# Center map
m = folium.Map(location=[data_cleaned['lat'].mean(), data_cleaned['long'].mean()], zoom_start=12)

palette = sns.color_palette("hsv", n_colors=data_cleaned['cluster_kmeans'].nunique())
colors = [mcolors.rgb2hex(color) for color in palette]

centroids = scaler.inverse_transform(kmeans.cluster_centers_)
sampled_data = data_cleaned.sample(n=20000, random_state=1)
cluster_counts = sampled_data['cluster_kmeans'].value_counts()

valid_clusters = cluster_counts[cluster_counts >= 3].index.tolist()

filtered_sampled_data = sampled_data[sampled_data['cluster_kmeans'].isin(valid_clusters)]
cluster_id_sample = filtered_sampled_data['cluster_kmeans'].unique().tolist()

print(cluster_id_sample) 

# Filter points based on distance to centroids
filtered_points = filtered_sampled_data[filtered_sampled_data.apply(
    lambda row: any(geodesic((row['lat'], row['long']), centroid).m <= 300 for centroid in centroids), axis=1)]

# Map markers
for i, row in filtered_points.iterrows():
    point = (row['lat'], row['long'])
    folium.CircleMarker(
        location=point,
        radius=1,
        color=colors[row['cluster_kmeans']],
        fill=True,
        fill_color=colors[row['cluster_kmeans']]
    ).add_to(m)

# Centroids and hulls
for cluster_id in cluster_id_sample:
    cluster_points = filtered_points[filtered_points['cluster_kmeans'] == cluster_id][['lat', 'long']].values
    if len(cluster_points) >= 3:  # ConvexHull requires at least 3 points
        try: 
            hull = ConvexHull(cluster_points)
            hull_points = cluster_points[hull.vertices]
            folium.Polygon(
                locations=hull_points,
                color=colors[cluster_id],
                fill=True,
                fill_opacity=0.2
            ).add_to(m)
        except QhullError:
            print(f"Skipping cluster {cluster_id} due to QhullError")
    centroid = centroids[cluster_id]
    folium.Marker(
        location=[centroid[0], centroid[1]],
        popup=f'Centroid {cluster_id}',
    ).add_to(m)

m.save("../html/kmeans_map.html")

### Hierarchical Clustering

In [ ]:
df_clustering_without_duplicates = df_clustering.drop_duplicates() # Drop duplicates
print(f"Initial: {len(df_clustering)}")
print(f"After removing duplicates: {len(df_clustering_without_duplicates)}")
df_clustering_without_duplicates = df_clustering_without_duplicates.sample(n=10000, random_state=1) # Sample a subset of the data
scaler = StandardScaler()
scaled_data_duplicates = scaler.fit_transform(df_clustering_without_duplicates)
scaled_data_duplicates_df = pd.DataFrame(data=scaled_data_duplicates, columns=df_clustering.columns)

plt.figure(figsize=(10, 6))
plt.scatter(scaled_data_duplicates_df['lat'], scaled_data_duplicates_df['long'], alpha=0.5)
plt.title('Scaled Latitude and Longitude')
plt.xlabel('Scaled Latitude')
plt.ylabel('Scaled Longitude')
plt.grid(True)
plt.show()


In [ ]:
# Vérifier que 'lat' et 'long' sont bien présentes dans le DataFrame
if 'lat' not in df_clustering_without_duplicates.columns or 'long' not in df_clustering_without_duplicates.columns:
    raise ValueError("Les colonnes 'lat' et 'long' doivent être présentes dans le DataFrame.")

# Si 'hierarchical_cluster' n'existe pas, on applique le clustering
if 'hierarchical_cluster' not in df_clustering_without_duplicates.columns:
    hierarchical_cluster = AgglomerativeClustering(n_clusters=20)  # Ajustez le nombre de clusters
    df_clustering_without_duplicates['hierarchical_cluster'] = hierarchical_cluster.fit_predict(df_clustering_without_duplicates[['lat', 'long']])

# Palette de couleurs
palette = sns.color_palette("hsv", n_colors=df_clustering_without_duplicates['hierarchical_cluster'].nunique())
colors = [mcolors.rgb2hex(color) for color in palette]

# Calculer les centroids manuellement pour chaque cluster
centroids = df_clustering_without_duplicates.groupby('hierarchical_cluster')[['lat', 'long']].mean().values

# Centrer la carte
m = folium.Map(location=[df_clustering_without_duplicates['lat'].mean(), df_clustering_without_duplicates['long'].mean()], zoom_start=12)

# Filtrer les points basés sur la distance aux centroids
filtered_points = df_clustering_without_duplicates[df_clustering_without_duplicates.apply(
    lambda row: any(geodesic((row['lat'], row['long']), centroid).m <= 200 for centroid in centroids), axis=1)]

# Ajouter des points filtrés à la carte
for i, row in filtered_points.iterrows():
    point = (row['lat'], row['long'])
    cluster_id = int(row['hierarchical_cluster'])  # Convertir en entier
    folium.CircleMarker(
        location=point,
        radius=1,
        color=colors[cluster_id],  # Utiliser 'hierarchical_cluster' pour la couleur
        fill=True,
        fill_color=colors[cluster_id]
    ).add_to(m)

# Dessiner les Convex Hulls et les centroids
for cluster_id in range(df_clustering_without_duplicates['hierarchical_cluster'].nunique()):
    cluster_points = filtered_points[filtered_points['hierarchical_cluster'] == cluster_id][['lat', 'long']].values
    if len(cluster_points) >= 3:  # ConvexHull nécessite au moins 3 points
        try:
            hull = ConvexHull(cluster_points)
            hull_points = cluster_points[hull.vertices]
            folium.Polygon(
                locations=hull_points,
                color=colors[cluster_id],
                fill=True,
                fill_opacity=0.2
            ).add_to(m)
        except Exception as e:
            print(f"Erreur avec le cluster {cluster_id}: {e}")

    centroid = centroids[cluster_id]
    folium.Marker(
        location=[centroid[0], centroid[1]],
        popup=f'Centroid {cluster_id}',
    ).add_to(m)

# Sauvegarder la carte
m.save("../html/hierarchical_map.html")


## DBSCAN

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
def plot_k_distance_graph(X, k):
    neigh = NearestNeighbors(n_neighbors=k)
    neigh.fit(X)
    distances, _ = neigh.kneighbors(X)
    distances = np.sort(distances[:, k-1])
    plt.figure(figsize=(10, 10))
    plt.plot(distances)
    plt.xlabel('Points')
    plt.ylabel(f'{k}-th nearest neighbor distance')
    plt.title('K-distance Graph')

    y_min, y_max = 0, 0.1
    y_ticks = np.arange(y_min, y_max, 0.01)  # Création des ticks de 0.001 en 0.001
    plt.yticks(y_ticks)
    plt.grid(axis='y', which='both', linestyle='--', linewidth=0.5)
    plt.show()
# Plot k-distance graph
plot_k_distance_graph(scaled_data_df, k=5)

In [ ]:


# Sélection des features pour le clustering
features = data_cleaned[columns].drop_duplicates()

# Normalisation des données complètes
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Appliquer DBSCAN
dbscan = DBSCAN(eps=0.015, min_samples=50)
clusters = dbscan.fit_predict(scaled_features)

# Ajouter les clusters aux données
features['cluster_dbscan'] = clusters
#data_cleaned['cluster_dbscan'] = clusters
# Afficher la distribution des clusters


In [ ]:

def gen_destination(number_of_points, features):
    """
    Sélectionne aléatoirement un nombre donné de centroïdes à partir des clusters DBSCAN.
    
    :param number_of_points: Nombre de centroïdes à sélectionner.
    :param features: DataFrame contenant les données avec la colonne 'cluster_dbscan'.
    :return: Numpy array des centroïdes sélectionnés.
    """
    clusters = [c for c in features['cluster_dbscan'].unique() if c != -1]  # Exclure les outliers (-1)
    
    if not clusters:
        raise ValueError("Aucun cluster valide trouvé dans les données.")

    cluster_centroids = []
    
    for cluster in clusters:
        cluster_data = features[features['cluster_dbscan'] == cluster].drop(columns=['cluster_dbscan'])
        centroid = cluster_data.mean().to_numpy()
        cluster_centroids.append(centroid)

    # Vérification du nombre de points demandés
    max_points = len(cluster_centroids)
    if number_of_points > max_points:
        raise ValueError(f"Le nombre de points demandés ({number_of_points}) est supérieur au nombre de clusters disponibles ({max_points}).")

    # Sélectionner aléatoirement le nombre de points demandés
    selected_centroids = random.sample(cluster_centroids, number_of_points)

    return np.array(selected_centroids)


# Exemple d'utilisation
nbr = int(input("Enter the number of destinations you want to visit: "))
destination = gen_destination(nbr, features)
print(destination)


In [ ]:
def tsp_solver_brute_force(points):
    """
    Résolution exacte du TSP avec une approche brute-force (permutation de tous les chemins possibles).
    
    :param points: Numpy array des coordonnées des points.
    :return: Numpy array des coordonnées des points dans l'ordre optimal.
    """
    num_points = len(points)
    if num_points <= 1:
        return points
    
    distance_matrix = cdist(points, points, metric='euclidean')  # Matrice des distances
    
    best_route = None
    best_distance = float('inf')
    
    for perm in permutations(range(num_points)):  # Générer toutes les permutations possibles
        distance = sum(distance_matrix[perm[i], perm[i + 1]] for i in range(num_points - 1))
        if distance < best_distance:
            best_distance = distance
            best_route = perm
    
    return points[np.array(best_route, dtype=int)]  # Retourner les coordonnées dans l'ordre optimal

# Application avec les données fournies

optimized_coordinates_tsp = tsp_solver_brute_force(destination)
print(optimized_coordinates_tsp)

In [ ]:
# Créer une carte centrée sur la moyenne des coordonnées
map_center = [optimized_coordinates_tsp[:, 0].mean(), optimized_coordinates_tsp[:, 1].mean()]

m = folium.Map(location=[features['lat'].mean(), features['long'].mean()], zoom_start=12)

# Filtrer les clusters valides (exclure le bruit: cluster -1)
valid_clusters = features['cluster_dbscan'][features['cluster_dbscan'] != -1].unique()
palette = sns.color_palette("hsv", n_colors=len(valid_clusters))
colors = {cluster: mcolors.rgb2hex(color) for cluster, color in zip(valid_clusters, palette)}

# Sous-échantillonnage


# Ajouter les points de clusters à la carte
for i, row in features.iterrows():
    cluster_id = row['cluster_dbscan']
    if cluster_id == -1:  # Ignorer le bruit
        continue
    folium.CircleMarker(
        location=(row['lat'], row['long']),
        radius=1,
        color=colors[cluster_id],
        fill=True,
        fill_color=colors[cluster_id]
    ).add_to(m)

# Calculer les barycentres des clusters pour les représenter comme "centroïdes"
for cluster_id in valid_clusters:
    cluster_points = features[features['cluster_dbscan'] == cluster_id][['lat', 'long']].values
    
    if len(cluster_points) == 0:
        continue
    
    # Barycentre du cluster
    centroid = cluster_points.mean(axis=0)
    
    # Ajouter un marqueur pour le barycentre
    folium.Marker(
        location=[centroid[0], centroid[1]],
        popup=f'Cluster {cluster_id}',
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

    # Dessiner l'enveloppe convexe si au moins 3 points
    if len(cluster_points) >= 3:
        try:
            hull = ConvexHull(cluster_points)
            hull_points = cluster_points[hull.vertices]
            folium.Polygon(
                locations=hull_points,
                color=colors[cluster_id],
                fill=True,
                fill_opacity=0.2
            ).add_to(m)
        except QhullError:
            print(f"Skipping cluster {cluster_id} due to QhullError")
    
graph = ox.graph_from_point(map_center, dist=5000, network_type='drive')

# Trouver les nœuds les plus proches des points optimisés sur le réseau routier
nearest_nodes = [ox.distance.nearest_nodes(graph, long, lat) for lat, long in optimized_coordinates_tsp]

# Ajouter les points optimisés sur la carte
for i, coord in enumerate(optimized_coordinates_tsp):
    folium.Marker(
        location=[coord[0], coord[1]],
        popup=f'Point {i+1}',
        icon=folium.Icon(color="green", icon="info-sign")
    ).add_to(m)

# Tracer l'itinéraire routier entre les points
route_coords = []
for i in range(len(nearest_nodes) - 1):
    # Calculer le plus court chemin entre les nœuds
    route = nx.shortest_path(graph, nearest_nodes[i], nearest_nodes[i+1], weight="length")
    # Obtenir les coordonnées des routes
    route_coords.extend([(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in route])

# Ajouter la route optimisée sur la carte
folium.PolyLine(
    locations=route_coords,
    color="red",
    weight=2.5,
    opacity=1
).add_to(m)


# Sauvegarder la carte
m.save("../html/dbscan_map.html")

## Text mining

### Preprocessing

In [ ]:
import nltk
import re
from nltk.corpus import stopwords


# Ensure NLTK stopwords are downloaded.
nltk.download('stopwords')

# Combine English and French stopwords.
stopwords_en = set(stopwords.words('english'))
stopwords_fr = set(stopwords.words('french'))
all_stopwords = stopwords_en.union(stopwords_fr)

def clean_tags(tag_string, words_to_remove):
    """
    Processes a comma-separated string of tags:
      - Splits the string on commas.
      - Removes any tag that is a stopword, contains a digit, or is in the custom removal list.
      - Returns the cleaned tags as a comma-separated string.
    """
    # Split the string on commas and remove extra whitespace.
    raw_tags = [tag.strip() for tag in tag_string.split(',') if tag.strip()]
    processed_tags = []
    
    for tag in raw_tags:
        # Remove all internal whitespace
        tag = re.sub(r'\s+', '', tag)
        
        # Special handling: if the tag starts with "uploaded:by=",
        # then remove that prefix and keep only the value (e.g., "instagram").
        if tag.lower().startswith("uploaded:by="):
            tag = tag[len("uploaded:by="):]
        
        processed_tags.append(tag)
    
    # Filter out tags that are stopwords, contain digits, or are in the additional removal list.
    filtered_tags = [
        tag for tag in processed_tags
        if tag.lower() not in all_stopwords
           and not re.search(r'\d', tag)
           and tag.lower() not in words_to_remove
    ]
    
    return ','.join(filtered_tags)

def clean_titles(title_string, words_to_remove):
    """
    Processes a title string:
      - Splits the title on whitespace.
      - Removes any word that is a stopword, contains a digit, or is in the custom removal list.
      - Returns the cleaned title as a space-separated string.
    """
    words = [word.strip() for word in title_string.split() if word.strip()]
    filtered_words = [
        word for word in words
        if word.lower() not in all_stopwords
           and not re.search(r'\d', word)
           and word.lower() not in words_to_remove
    ]
    return ' '.join(filtered_words)

In [ ]:
words_to_remove = {'iphone', 'iphonex', 'instagram', 'upload', 'rhone', 
                    'rhonealpes', 'lyon', 'france', 'rhônealpes'}


if 'tags' in data_cleaned.columns:
    data_cleaned['tags'] = data_cleaned['tags'].apply(
        lambda x: clean_tags(x, words_to_remove) if isinstance(x, str) else x
    )
    # Remove rows where 'tags' becomes empty after cleaning.
    data_cleaned = data_cleaned.dropna(subset=['tags'])
    data_cleaned = data_cleaned[data_cleaned['tags'].str.strip() != '']

if 'title' in data_cleaned.columns:
    data_cleaned['title'] = data_cleaned['title'].apply(
        lambda x: clean_tags(x, words_to_remove) if isinstance(x, str) else x
    )
    # Remove rows where 'titles' becomes empty after cleaning.
    data_cleaned = data_cleaned.dropna(subset=['title'])
    data_cleaned = data_cleaned[data_cleaned['title'].str.strip() != '']

data_cleaned.to_csv("../data/text-mining.csv", index=False)
print("Data exported to 'text-mining.csv'")

### Word Cloud Visualisation

In [ ]:
from wordcloud import WordCloud

import matplotlib.pyplot as plt

all_tags = ' '.join(data_cleaned['tags'].dropna())

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_tags)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Tags')
plt.show()


We could not that some text are not relevant for tagging purposes, ie. instagramapp, square, squareformat, etc.
We could then proceed to remove some more data.

In [ ]:
words_to_remove = {'square', 'squareformat', 'instagramapp', 'uploaded', 'iphoneography', 'europe', 'rhône'}


if 'tags' in data_cleaned.columns:
    data_cleaned['tags'] = data_cleaned['tags'].apply(
        lambda x: clean_tags(x, words_to_remove) if isinstance(x, str) else x
    )
    # Remove rows where 'tags' becomes empty after cleaning.
    data_cleaned = data_cleaned.dropna(subset=['tags'])
    data_cleaned = data_cleaned[data_cleaned['tags'].str.strip() != '']


We then revisualise again, these procedure can be done iteratively.

In [ ]:
all_tags = ' '.join(data_cleaned['tags'].dropna())

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_tags)

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Tags')
plt.show()

## Term frequency and inverse document frequency (TF-IDF)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def get_cluster_top_terms(cluster_id, data, cluster_col='cluster_kmeans', text_col='tags', top_n=10):
    """
    Pour un cluster donné, cette fonction :
      - Filtre les lignes du DataFrame pour ce cluster.
      - Agrège le contenu textuel de la colonne spécifiée (par exemple, 'tags').
      - Calcule la TF-IDF en utilisant les stopwords français.
      - Retourne une liste des top_n termes triés par leur score TF-IDF.
    
    Parameters:
        cluster_id : Identifiant du cluster.
        data       : DataFrame contenant les données.
        cluster_col: Nom de la colonne contenant les labels de cluster.
        text_col   : Nom de la colonne contenant le texte.
        top_n      : Nombre de termes à extraire.
    
    Returns:
        list: Liste des top_n termes pour le cluster.
    """
    cluster_data = data[data[cluster_col] == cluster_id]
    if cluster_data.empty:
        return []
    
    aggregated_text = " ".join(cluster_data[text_col].astype(str).tolist())
    
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([aggregated_text])
    feature_names = vectorizer.get_feature_names_out()
    scores = tfidf_matrix.toarray().flatten()
    
    if len(scores) < top_n:
        top_n = len(scores)
    top_indices = scores.argsort()[-top_n:][::-1]
    top_terms = [feature_names[i] for i in top_indices]
    
    return top_terms


In [ ]:
unique_clusters = sorted(data_cleaned['cluster_kmeans'].unique()) # For cluster kmeans

for cluster_id in unique_clusters:
    terms = get_cluster_top_terms(cluster_id, data_cleaned, cluster_col='cluster_kmeans', text_col='tags', top_n=10)
    print(f"Top terms for cluster {cluster_id}: {terms}")

## Events

We first define the events for Lyon in a dictionary, this dictionary can of course be expanded further but for the sake of simplicity, we've reduced to some of the major events in Lyon.

In [ ]:
events_in_lyon = {
    "Fête des Lumières": {year: ((12, 8), (12, 11)) for year in range(2005, 2020)},
    "Nuits Sonores": {
        2005: ((5, 12), (5, 14)),
        2006: ((5, 11), (5, 13)),
        2007: ((5, 10), (5, 12)),
        2008: ((5, 10), (5, 12)),
        2009: ((5, 9),  (5, 11)),
        2010: ((5, 9),  (5, 11)),
        2011: ((5, 9),  (5, 11)),
        2012: ((5, 9),  (5, 11)),
        2013: ((5, 9),  (5, 11)),
        2014: ((5, 9),  (5, 11)),
        2015: ((5, 9),  (5, 11)),
        2016: ((5, 9),  (5, 11)),
        2017: ((5, 9),  (5, 11)),
        2018: ((5, 9),  (5, 11)),
        2019: ((5, 9),  (5, 11))
    },
    "Fête de la Musique": {year: ((6, 21), (6, 21)) for year in range(2005, 2020)},
    "Les Nuits de Fourvière": {year: ((7, 1), (7, 15)) for year in range(2005, 2020)}
}

We first construct a datetime column from the year, month, and day columns by extracting the corresponding day-of-year and year (filtered to 2005–2019). 

Then, we aggregates the daily photo counts and applies a 7-day rolling mean to smooth out short-term fluctuations — this period is chosen because it captures a full week’s activity, which helps reveal underlying trends and recurring patterns while reducing noise.

We then use peak detection (based on the 90th percentile threshold) to identify days with unusually high photo counts.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks
import datetime
import os

def event_from_day_of_year(day_of_year, reference_year):
    """
    Convert a day-of-year (1-366) into a (month, day) tuple using the given reference year,
    then check if it falls within any known event date ranges for that year.
    """
    dt = datetime.datetime(reference_year, 1, 1) + datetime.timedelta(days=day_of_year - 1)
    current_month, current_day = dt.month, dt.day
    for event, event_dates in events_in_lyon.items():
        # If event_dates is a dict, look up the range for the reference_year.
        if isinstance(event_dates, dict):
            if reference_year in event_dates:
                (start_month, start_day), (end_month, end_day) = event_dates[reference_year]
            else:
                continue  # No date info for this event in this year.
        else:
            (start_month, start_day), (end_month, end_day) = event_dates
        start_dt = datetime.datetime(reference_year, start_month, start_day)
        end_dt = datetime.datetime(reference_year, end_month, end_day)
        if start_dt <= dt <= end_dt:
            return event
    return None


# Create a datetime column using the year, month, and day columns
data_cleaned['date_taken'] = pd.to_datetime(
    data_cleaned['date_taken_year'].astype(str) + '-' +
    data_cleaned['date_taken_month'].astype(str) + '-' +
    data_cleaned['date_taken_day'].astype(str),
    errors='coerce'
)

# Use the provided year column
data_cleaned['year'] = data_cleaned['date_taken_year']
data_cleaned['day_of_year'] = data_cleaned['date_taken'].dt.dayofyear

years = sorted([y for y in data_cleaned['year'].dropna().unique() if 2005 <= y <= 2019]) # Limit to 2005-2019
print("Years considered:", years)

n_years = len(years)
fig, axes = plt.subplots(n_years, 1, figsize=(12, 4 * n_years), sharex=True)
if n_years == 1:
    axes = [axes]

peak_info = {}  # To store detected peaks and associated events for each year

for i, yr in enumerate(years):
    yearly_data = data_cleaned[data_cleaned['year'] == yr]
    # Compute daily photo counts by day-of-year
    daily_counts = yearly_data.groupby('day_of_year').size()
    
    # Apply a 7-day rolling mean to smooth the counts
    daily_counts_smoothed = daily_counts.rolling(window=7, center=True).mean()
    
    # Work only with non-NaN values
    smoothed = daily_counts_smoothed.dropna()
    
    if smoothed.empty:
        print(f"No data available for year {yr}")
        ax = axes[i]
        ax.set_title(f"Photo Activity in {yr} (No Data)")
        continue
    
    # Detect peaks above the 90th percentile threshold
    threshold = np.percentile(smoothed, 90)
    peaks, properties = find_peaks(smoothed, height=threshold)
    
    # Use an empty list if no peaks are detected
    if len(peaks) == 0:
        detected_peak_days = []
    else:
        detected_peak_days = smoothed.index[peaks]
    
    # Associate each detected peak with a known event for that year
    peak_events = {}
    for peak_day in detected_peak_days:
        ev = event_from_day_of_year(peak_day, int(yr))
        peak_events[peak_day] = ev
    peak_info[yr] = peak_events
    
    ax = axes[i]
    ax.plot(daily_counts.index, daily_counts, label='Daily Photo Counts', alpha=0.5)
    ax.plot(smoothed.index, smoothed, label='7-Day Rolling Mean', color='red')
    if len(peaks) > 0:
        ax.scatter(smoothed.index[peaks], smoothed.iloc[peaks], 
                   color='green', marker='x', s=100, label='Detected Peaks')
    ax.set_title(f"Photo Activity in {yr}")
    ax.set_xlabel("Day of Year")
    ax.set_ylabel("Number of Photos")
    ax.legend()

plt.tight_layout()
plt.show()

These peaks can then be mapped to the dictionary defined earlier. An export of matching events can be found in output folder.

In [ ]:
import os
import datetime

for yr in years:
    printed_any = False
    for peak_day, event in peak_info.get(yr, {}).items():
        if event is not None:
            dt = datetime.datetime(int(yr), 1, 1) + datetime.timedelta(days=peak_day - 1)
            date_str = dt.strftime("%B %d")  # e.g., "December 09"
            print(f"Year {yr} - Day {peak_day} ({date_str}) -> {event}")
            printed_any = True
    if not printed_any:
        print(f"Year {yr}: No matching events")

output_dir = "../output"
os.makedirs(output_dir, exist_ok=True)
events_list = []
for yr in years:
    for peak_day, event in peak_info.get(yr, {}).items():
        if event is not None:  # Only export if there's a matching event
            dt = datetime.datetime(int(yr), 1, 1) + datetime.timedelta(days=peak_day - 1)
            date_str = dt.strftime("%Y-%m-%d")
            events_list.append({
                "year": yr,
                "day_of_year": peak_day,
                "date": date_str,
                "event": event
            })

events_df = pd.DataFrame(events_list)
output_path = os.path.join(output_dir, "events.csv")
events_df.to_csv(output_path, index=False)
print(f"Event data exported to {output_path}")


We can then conclude that the peaks of photo taken can be justified with events taking places at Lyon, especially Fête des Lumières.